In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os

In [71]:
# url = 'https://www.nldc.evn.vn/api/services/app/Pages/GetChartNguonDien'
# #get data from url
# response = requests.get(url)
# #convert data to json
# data = response.json()
# #convert data to pandas dataframe
# df = pd.DataFrame(data['result']['data']['nguonDiens'])
# df

# 1. SCS

In [2]:
SCS = pd.DataFrame()
def get_SCS(i):
    url = 'https://ecargo.scsc.vn/General/Tonnage/Public'
    payload = {'year':i}
    response = requests.post(url, data=payload)
    data = response.json()
    df = pd.DataFrame(data)
    return df
for i in range(2017,2025):
    x = get_SCS(i)
    SCS = pd.concat([SCS,x])

In [3]:
SCS.to_excel('C:/Users/HP PAVILION/Desktop/data/IR/IR/SCS/SCS.xlsx')

# 2. TLG

In [6]:
def get_TLG_VI(x):
    response = requests.get(x)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = []
    pdfs = []
    for link in soup.find_all('a'):
        if 'nhà đầu tư' in link.text.lower():
            titles.append(link.text)
            if link.get('href').endswith('.pdf'):
                pdfs.append(link.get('href'))
    #create a dataframe from titles and pdfs
    df = pd.DataFrame({'Title':titles, 'PDF':pdfs})
    return df

def get_TLG_EN(x):
    response = requests.get(x)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = []
    pdfs = []
    for link in soup.find_all('a'):
        if 'newsletter' in link.text.lower():
            titles.append(link.text)
            if link.get('href').endswith('.pdf'):
                pdfs.append(link.get('href'))
    #create a dataframe from titles and pdfs
    df = pd.DataFrame({'Title':titles, 'PDF':pdfs})
    return df



In [7]:
TLG_VI = pd.DataFrame()
TLG_EN = pd.DataFrame()
for i in range(1,19):
    url_VI = 'https://thienlonggroup.com/quan-he-co-dong/tai-lieu-khac?page='+str(i)
    url_EN = 'https://thienlonggroup.com/en/quan-he-co-dong/tai-lieu-khac?page='+str(i)
    #use get_TLG function to get data from url and append to dataframe TLG
    x = get_TLG_VI(url_VI)
    y = get_TLG_EN(url_EN)
    TLG_VI = pd.concat([TLG_VI,x])
    TLG_EN = pd.concat([TLG_EN,y])


In [8]:
#get desktop dir
desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
print(desktop_path)

C:\Users\HP PAVILION\Desktop


In [11]:
#save pdf from PDF to C:/Users/hoang.nguyennhat/Desktop/IR/TLG with Title as file name
for i in range(len(TLG_VI)):
    response = requests.get(TLG_VI['PDF'].iloc[i])
    with open(desktop_path+'/IR/TLG/VI/'+TLG_VI['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)
for i in range(len(TLG_EN)):
    response = requests.get(TLG_EN['PDF'].iloc[i])
    with open(desktop_path+'/IR/TLG/EN/'+TLG_VI['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\HP PAVILION\\DesktopIR/TLG/VI/Bản tin dành cho Nhà đầu tư Tháng 2 2024.pdf'

In [ ]:
#get all the filename from C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/
import os
files = os.listdir('C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/')
#select the string before - of the filename, remove blank space
a = [x.split('-')[0].strip() for x in files]
#remove .pdf from the string
a = [x.replace('.pdf','') for x in a]
#convert files to format YYYY-mm, for example 'August 2021' to '2021-08'
a = [pd.to_datetime(x).strftime('%Y-%m') for x in a]


In [ ]:
#replace the file in files with list a
for i in range(len(files)):
    os.rename('C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/'+files[i],'C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/'+a[i]+'.pdf')

# 3. HDG

In [4]:
list_id = []
list_file = []
url = 'https://hado.com.vn/Ajax/Home/LoadContentShareHolders'
def get_id_HDG(i):
    data = {'p': i,'id': 19}
    response = requests.post(url, data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    #find table in soup
    table = soup.find('table')
    #convert table to dataframe
    df = pd.read_html(str(table))[0]
    #find javascript void in soup
    for link in soup.find_all('a'):
        #select the string include void and get the number in the ()
        if 'javascript:void' in link.get('href'):
            
        #add it into list_id
            list_id.append(link.get('href').split('(')[1].split(')')[0])
            list_file.append(link.text)
        #create the dataframe from list_id and list_file
    df = pd.DataFrame({'ID':list_id, 'File':list_file})
    return df

In [5]:
def get_pdf_HDG(id):
    url = 'https://hado.com.vn/Ajax/Content/ViewDetail?id='+str(id)
    base_url = 'https://hado.com.vn'
    response = requests.get(url)
    #response json
    data = response.json()
    #find href in data
    soup = BeautifulSoup(data['content'], 'html.parser')
    #find all href
    for link in soup.find_all('a'):
        a = (base_url+link.get('href'))
    return a

In [6]:
df = pd.DataFrame()
for i in range(1,5):
    x = get_id_HDG(i)
    df = pd.concat([df,x])

In [7]:
df['PDF'] = df['ID'].apply(get_pdf_HDG)

In [18]:
df.iloc[0,2]

'https://hado.com.vn/Uploads/files/BAN%20TIN%20IR%20QUY%201%202024%20(1).pdf'

In [34]:
#use tnrange to create progress bar
from tqdm import tnrange
#replace / with - in File
df['File'] = df['File'].str.replace('/','-')
#save pdf to C:/Users/hoang.nguyennhat/Desktop/IR/HDG
for i in tnrange(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open('C:/Users/hoang.nguyennhat/Desktop/IR/HDG/'+str(df['File'].iloc[i])+'.pdf', 'wb') as f:
        f.write(response.content) 

C:\Users\hoang.nguyennhat\AppData\Local\Temp\ipykernel_29068\485858271.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(df)):


  0%|          | 0/93 [00:00<?, ?it/s]

In [68]:
import os
#get the path of desktop from os
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#get files from desktop/data/IR
files = os.listdir(desktop+'/data/IR/IR/HDG/')
#get the string after tư
a = [x.split('tư')[1].strip() for x in files]
a = [x.lower().replace('tháng ','T').replace('quý ','Q').strip() for x in a]
#get the string after - and before .pdf in a and change it to the beginning of the string
a = [x.split('-')[1].replace('.pdf','')+'-'+x.split('-')[0]+'-HDG.pdf' for x in a]
a = [x.replace('Qi-','Q1-').replace('Qii-','Q2-').replace('Qiii-','Q3-').replace('Qiv-','Q4-') for x in a]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/HDG/'+files[i],desktop+'/data/IR/IR/HDG/'+a[i])

# 4. PGV

In [19]:
base_url = 'https://www.genco3.com'
def get_pdf_PGV(x):
    response = requests.get(x)
    soup = BeautifulSoup(response.text, 'html.parser')
    list_file = []
    list_name = []
    #file all div class = 'file' in soup
    for link in soup.find_all('div', class_='sn-new-item'):
    #find p class = 'title'
        name = link.find('p', class_='title').text
        #beautify the name
        name = name.replace('\n','').strip()
        list_name.append(name)
        #get pdf link from a tag
        pdf = link.find('a').get('href')
        pdf = base_url+pdf
        list_file.append(pdf)
    #convert list_name and list_file to dataframe
    df = pd.DataFrame({'Name':list_name, 'PDF':list_file})
    #find name include BẢN TIN in Name
    df = df[df['Name'].str.lower().str.contains('tin')]
    #replace / with - in Name
    df['Name'] = df['Name'].str.replace('/','-')
    return df

In [20]:
url = 'https://www.genco3.com/quan-he-nha-dau-tu/ban-tin-nha-dau-tu.html?y=2024'
df = get_pdf_PGV(url)


In [22]:
df.iloc[0,1]

'https://www.genco3.com/data/files/20240522/638519722466716673_0.pdf'

In [ ]:
#save pdf to C:/Users/hoang.nguyennhat/Desktop/IR/PGV
for i in tnrange(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open('C:/Users/hoang.nguyennhat/Desktop/IR/PGV/'+str(df['Name'].iloc[i])+'.pdf', 'wb') as f:
        f.write(response.content)

In [ ]:
#rename the file in C:/Users/hoang.nguyennhat/Desktop/IR/PGV to format YYYY-mm, for example 'BẢN TIN NHÀ ĐẦU TƯ THÁNG 01 NĂM 2020' to '2020-01-PGV'
files = os.listdir('C:/Users/hoang.nguyennhat/Desktop/IR/PGV/')
a = [x.split('NĂM ')[1].split(' ')[0] for x in files]
a = [pd.to_datetime(x).strftime('%Y-%m') for x in a]
for i in range(len(files)):
    os.rename('C:/Users/hoang.nguyennhat/Desktop/IR/PGV/'+files[i],'C:/Users/hoang.nguyennhat/Desktop/IR/PGV/'+a[i]+'-PGV.pdf')
    

# 5. MWG

In [25]:
def get_link_MWG(x,i):
    payload = {'langID': 'vi-VN','cateID': i}
    response = requests.get(x, params=payload)
    response = response.json()['result']
    soup = BeautifulSoup(response, 'html.parser')
    titles = []
    pdfs = []
    for link in soup.find_all('a'):
        if link.get('href').endswith('.pdf'):
            pdf = link.get('href')
            pdfs.append(pdf)
            #get title from link text, remove blank space and \n
            title = link.text.replace('\n','').strip()
            titles.append(title)
    #create dataframe from title and pdf
    df = pd.DataFrame({'Title':titles, 'PDF':pdfs})
    return df


In [26]:
list_cateID = [193,185,106,98,5,6,28,27,26,25]
url = 'https://mwg.vn/Category/GetReportByYear?langID=vi-VN&cateID='
df = pd.DataFrame()
for i in list_cateID:
    a = url+str(i)
    x = get_link_MWG(a,i)
    df = pd.concat([df,x])
df['Title'] = df['Title'].str.replace('|','-')
df['Title'] = df['Title'].str.replace('/','-')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#exclude 'Báo cáo tài chính', 'Báo cáo quản trị', 'Báo cáo thường niên' from Title lowercase
df = df[~df['Title'].str.lower().str.contains('báo cáo tài chính')]
df = df[~df['Title'].str.lower().str.contains('báo cáo quản trị')]
df = df[~df['Title'].str.lower().str.contains('báo cáo thường niên')]
#get desktop path
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to C:/Users/hoang.nguyennhat/Desktop/IR/MWG/VI with Title as file name
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/MWG/VI/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)


In [ ]:
files = os.listdir(desktop+'/data/IR/IR/MWG/VI/')
#select the character before .pdf in files
a = [x.split('.pdf')[0] for x in files]
#select last word of a, and move it to the beginning of the string
a = [x.split(' ')[-1]+'-'+x+'.pdf' for x in a]
a = [x.replace('1-2','2').replace('2-2','2').replace('3-2','2').replace('4-2','2').replace('năm -','2023') for x in a]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/MWG/VI/'+files[i],desktop+'/data/IR/IR/MWG/VI/'+a[i])


In [14]:
#EN
list_cateID = [196,188,110,101,65,66,67,68]
url = 'https://mwg.vn/Category/GetReportByYear?langID=en-US&cateID='
df = pd.DataFrame()
for i in list_cateID:
    a = url+str(i)
    x = get_link_MWG(a,i)
    df = pd.concat([df,x])
df['Title'] = df['Title'].str.replace('|','-')
df['Title'] = df['Title'].str.replace('/','-')

In [17]:
df = df[~df['Title'].str.lower().str.contains('fs')]
df = df[~df['Title'].str.lower().str.contains('governance')]
df = df[~df['Title'].str.lower().str.contains('sustainability')]

In [19]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to C:/Users/hoang.nguyennhat/Desktop/IR/MWG/VI with Title as file name
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/MWG/EN/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

# 6. VJC

In [27]:
url = 'https://ir.vietjetair.com/Home/Menu/ban-tin-nha-dau-tu'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
#find all pdf links in soup
pdfs = []
title = []
base_url = 'https://ir.vietjetair.com'
for link in soup.find_all('a'):
    if link.get('href').endswith('.pdf'):
        pdfs.append(link.get('href'))
        title.append(link.text)
#convert pdfs to dataframe
df = pd.DataFrame({'PDF':pdfs, 'Title':title})
df['PDF'] = base_url+df['PDF']
df['Title'] = df['Title'].str.replace('/','-')


In [33]:
df = df.iloc[0:4,:]

In [35]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to desktop/data/IR/IR/VJC with Title as file name
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/VJC/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

In [38]:
files = os.listdir(desktop+'/data/IR/IR/VJC/')
#select the last 4 character before .pdf
a = [x[-8:-4] for x in files]
#select the 2 character after tháng
b = [x.lower().split('tháng ')[1][:2] for x in files]
#select a and b and move it to the beginning of the string
a = [x+'-'+y+'-'+'VJC'+'.pdf' for x,y in zip(a,b)]
#replace 1--,2--,3--,4--,5--,6--,7--,8--,9-- with 01-,02-,03-,04-,05-,06-,07-,08-,09-
a = [x.replace('1--','01-').replace('2--','02-').replace('3--','03-').replace('4--','04-').replace('5--','05-').replace('6--','06-').replace('7--','07-').replace('8--','08-').replace('9--','09-') for x in a]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/VJC/'+files[i],desktop+'/data/IR/IR/VJC/'+a[i])

# 7. CTG

In [36]:
url = 'https://investor.vietinbank.vn/OtherDocuments.aspx'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
pdfs = []
titles = []
tr = soup.find_all('tr')
for i in tr:
    #find a in tr
    a = i.find_all('a')
    for j in a:
        #select the string include 'bản tin' in j.text
        if 'bản tin' in j.text.lower():
            #get pdf link from j
            pdf = j.get('href')
            #get title from j.text
            title = j.text
            #add pdf and title to list
            pdfs.append(pdf)
            titles.append(title)
#convert pdfs and titles to dataframe
df = pd.DataFrame({'PDF':pdfs, 'Title':titles})
df['PDF'] = 'https://investor.vietinbank.vn/'+df['PDF']
df['Title'] = df['Title'].str.replace('/','-')


In [6]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to desktop/data/IR/IR/CTG with Title as file name
for i in range(len(df)):
    # try with time out 20s
    try:
        response = requests.get(df['PDF'].iloc[i],timeout=20)
        with open(desktop+'/data/IR/IR/CTG/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
            f.write(response.content)
    except:
        print(df['PDF'].iloc[i])

https://investor.vietinbank.vn//Handlers/ViewReport.ashx?ReportID=1054
https://investor.vietinbank.vn//Handlers/ViewReport.ashx?ReportID=963
https://investor.vietinbank.vn//Handlers/ViewReport.ashx?ReportID=997
https://investor.vietinbank.vn//Handlers/ViewReport.ashx?ReportID=963


In [ ]:
files = os.listdir(desktop+'/data/IR/IR/CTG/')
#select the string after tháng and before .pdf in files, remove blank space
a = [x.lower().split('tháng ')[1].split('.pdf')[0].strip() for x in files]
a = [x[-4:]+'-'+x.split('-')[0]+'-CTG.pdf' for x in a]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/CTG/'+files[i],desktop+'/data/IR/IR/CTG/'+a[i])
    

# 8. PDR

In [38]:
def get_pdf_PDR(i):
    x = 'https://www.phatdat.com.vn/ban-tin-ir/?no='+str(i)
    payload = {'no': i}
    response = requests.get(x, params=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    pdfs = []
    titles = []
    #find all class = 'item' in soup
    for link in soup.find_all('div', class_='item'):
        for a in link.find_all('a'):
            pdf=(a.get('href'))
            pdfs.append(pdf)
            title=(link.find('h6', class_='title-report').text)
            titles.append(title)
    #convert pdfs and titles to dataframe
    df = pd.DataFrame({'PDF':pdfs, 'Title':titles})
    df['Title'] = df['Title'].str.replace('/','-')
    return df
        


In [39]:
df = pd.DataFrame()
for i in range(1,5):
    a = get_pdf_PDR(i)
    df = pd.concat([df,a])

In [42]:
df = df.drop_duplicates().iloc[:5,:]

In [43]:
#save pdf to desktop/data/IR/IR/PDR with Title as file name
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/PDR/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

In [61]:
files = os.listdir(desktop+'/data/IR/IR/PDR/')
b = [x.replace('Bản tin IR Quý','').strip() for x in files]
#select the last 4 character of b, put it in the beginning of the string
b = [x[-8:-4]+'-Q'+x[0]+'.pdf' for x in b]
#rename the file in files with list b
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/PDR/'+files[i],desktop+'/data/IR/IR/PDR/'+b[i]+'.pdf')


# 9. DIG

In [44]:
def get_link_DIG(i):
    url = 'https://www.dic.vn/ban-tin-nha-dau-tu?page='+str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    link = []
    titles = []
    
    #find div class = 'intro intro1' in soup
    for i in soup.find_all('div', class_='intro intro1'):
        #find a in i
        for j in i.find_all('a'):
            #get link from j
            link.append(j.get('href'))
            #get title from class = 'title' in i
            titles.append(i.find('a', class_='title').text)
    #convert link and titles to dataframe
    df = pd.DataFrame({'Title':titles,'Link':link})
    df['Link'] = 'https://www.dic.vn/'+df['Link']
    return df


In [45]:
df = pd.DataFrame()
for i in range(1,22):
    a = get_link_DIG(i)
    df = pd.concat([df,a])
df = df[df['Title'].str.lower().str.contains('ir')]

In [46]:
for i in range(len(df)):
    response = requests.get(df['Link'].iloc[i])
    soup = BeautifulSoup(response.text, 'html.parser')
    base_url = 'https://www.dic.vn/'
    #find all a in soup
    for link in soup.find_all('a'):
        #select the string include 'pdf' in link
        if 'pdf' in link.get('href'):
            #get pdf link from link
            x = base_url+(link.get('href'))
            df['PDF'].iloc[i] = x


KeyError: 'PDF'

In [89]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to desktop/data/IR/IR/DIG with Title as file name
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/DIG/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

In [97]:
files = os.listdir(desktop+'/data/IR/IR/DIG/')
#select the last 4 character before .pdf in files
a = [x[-8:-4] for x in files]
#select the 2 character after IR
b = [x.lower().split('ir')[1][:3].strip().upper() for x in files]
#rename a and b and move it to the beginning of the string
a = [x+'-'+y+'-DIG.pdf' for x,y in zip(a,b)]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/DIG/'+files[i],desktop+'/data/IR/IR/DIG/'+a[i])

# 10. DPM

In [48]:
def get_link_DPM(i):
    url = 'https://dpm.vn/category/ban-tin-nha-dau-tu/page/'+str(i)
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    link = []
    titles = []
    #find all h3 in soup
    for i in soup.find_all('h3'):
        #find a in i
        for j in i.find_all('a'):
            #get link from j
            link.append(j.get('href'))
            #get title from j
            titles.append(j.text)
    #convert link and titles to dataframe
    df = pd.DataFrame({'Title':titles,'Link':link})
    return df


In [49]:
df = pd.DataFrame()
for i in range(1,3):
    a = get_link_DPM(i)
    df = pd.concat([df,a])
df['PDF'] = 'NAN'
df['Title'] = df['Title'].str.replace('/','-')

c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-war

In [115]:
for i in range(len(df)):
    response = requests.get(df['Link'].iloc[i], verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    #find all a in soup
    for link in soup.find_all('a'):
        #select the string include 'pdf' in link
        if 'pdf' in link.get('href'):
            df['PDF'].iloc[i] = link.get('href')


c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-war

In [120]:
#save pdf to desktop/data/IR/IR/DPM with Title as file name
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i],verify=False)
    with open(desktop+'/data/IR/IR/DPM/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\HP PAVILION\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dpm.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-war

In [156]:
files = os.listdir(desktop+'/data/IR/IR/DPM/')
#select the last 4 character before .pdf in files
a = [x[-8:-4] for x in files]
#extract all number in the string
b = [x[:-8] for x in files]
#replace I with 1, II with 2, III with 3, IV with 4
b = [x.replace('I năm','1 năm').replace('II năm','2 năm').replace('III năm','3 năm').replace('IV năm','4') for x in b]
#extract all number from b using regular expression
import re
b = [re.findall(r'\d+', x) for x in b]
#convert b to string
b = [''.join(x) for x in b]
#rename the file in files with list a and b
a = [x+'-Q'+y+'-DPM.pdf' for x,y in zip(a,b)]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/DPM/'+files[i],desktop+'/data/IR/IR/DPM/'+a[i])


# 11. GAS

In [51]:
def get_link_GAS(i):
    url = 'https://www.pvgas.com.vn/DesktopModules/EasyDNNnews/ListContentHtml.ashx?portalId=0&moduleId=1514&tabId=153&contentKey=-1&contentPage='+str(i)+'&getPaginationMeta=false&_=1709751847694'
    response = requests.get(url)
    response = response.json()
    soup = BeautifulSoup(response['contentHtml'], 'html.parser')
    #find all a in soup
    links = []
    titles = []
    for link in soup.find_all('div', class_='row'):
        #find a in link
        for a in link.find_all('a'):
            if 'https' in a.get('href'):
                links.append(a.get('href'))
                titles.append(a.text.replace('\n','').strip())
    #convert pdfs and titles to dataframe
    df = pd.DataFrame({'Title':titles,'Link':links})
    df['Title'] = df['Title'].str.replace('/','-')
    return df



In [52]:
df = pd.DataFrame()
for i in range(1,3):
    a = get_link_GAS(i)
    df = pd.concat([df,a])
df['PDF'] = 'NAN'
df['Title'] = df['Title'].str.replace('/','-')
    

In [56]:
df = df.iloc[[0],:]

In [57]:
for i in range(len(df)):
    response = requests.get(df['Link'].iloc[i])
    soup = BeautifulSoup(response.text, 'html.parser')
    base_url = 'https://www.pvgas.com.vn/'
    #find li in soup with class = 'edn_docType_pdf'
    for link in soup.find_all('li', class_='edn_docType_pdf'):
        #find a in link
        for a in link.find_all('a'):
            #get pdf link from a
            pdf_link = base_url+a.get('href')
            df['PDF'].iloc[i] = pdf_link


C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_30044\2825293060.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PDF'].iloc[i] = pdf_link


In [58]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to desktop/data/IR/IR/GAS with Title as file name
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/GAS/'+df['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

In [205]:
files = os.listdir(desktop+'/data/IR/IR/GAS/')
#select the last 4 character before .pdf in files
a = [x[-8:-4] for x in files]
b = [x[:-8] for x in files]
#replace I with 1, II with 2, III with 3, IV with 4
b = [x.replace('I-','1-').replace('II-','2-').replace('III-','3-').replace('IV-','4') for x in b]
#extract all number from b using regular expression
import re
b = [re.findall(r'\d+', x) for x in b]
#convert b to string
b = [''.join(x) for x in b]
#rename the file in files with list a and b
a = [x+'-Q'+y+'-GAS.pdf' for x,y in zip(a,b)]
#rename the file in files with list a
for i in range(len(files)):
    os.rename(desktop+'/data/IR/IR/GAS/'+files[i],desktop+'/data/IR/IR/GAS/'+a[i])

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\HP PAVILION\\Desktop/data/IR/IR/GAS/Bản tin nhà đầu tư Quý II- 2020.pdf' -> 'C:\\Users\\HP PAVILION\\Desktop/data/IR/IR/GAS/2020-Q1-GAS.pdf'

# 12. PNJ

In [59]:
url = 'https://www.pnj.com.vn/quan-he-co-dong/bao-cao-thang/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
pdfs = []
#find all a in soup
for link in soup.find_all('a'):
    #select the string include 'pdf' in link
    if 'pdf' in link.get('href'):
        #get pdf link from link
        pdf = link.get('href')
        pdfs.append(pdf)
    
#convert pdfs and titles to dataframe
df = pd.DataFrame({'PDF':pdfs})
df

,PDF
0,https://cdn.pnj.io/images/quan-he-co-dong/2024...
1,https://cdn.pnj.io/images/quan-he-co-dong/2024...
2,https://cdn.pnj.io/images/quan-he-co-dong/2024...
3,https://cdn.pnj.io/images/quan-he-co-dong/2024...
4,https://cdn.pnj.io/images/quan-he-co-dong/2024...
...,...
104,https://cdn.pnj.io/images/quan-he-co-dong/2022...
105,https://cdn.pnj.io/images/quan-he-co-dong/2022...
106,https://cdn.pnj.io/images/quan-he-co-dong/2022...
107,https://cdn.pnj.io/images/quan-he-co-dong/2024...


In [60]:
#get last path after / in pdfs
df['Title'] = [x.split('/')[-1] for x in df['PDF']]

In [63]:
df = df.iloc[:5,:]

In [64]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
#save pdf to desktop/data/IR/IR/PNJ with Title as file name
for i in range(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open(desktop+'/data/IR/IR/PNJ/'+df['Title'].iloc[i], 'wb') as f:
        f.write(response.content)
    

In [255]:
response = requests.get(df['PDF'].iloc[i])
response.headers

{'Content-Type': 'application/pdf', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Wed, 06 Mar 2024 20:02:17 GMT', 'Last-Modified': 'Wed, 21 Feb 2024 01:08:29 GMT', 'ETag': 'W/"97442bdcb722e24787bb7c1ac7e49b5b"', 'x-amz-server-side-encryption': 'AES256', 'x-amz-version-id': 'XZDNmmBNLM8HpON33SDfWdoSrlkURNKf', 'Server': 'AmazonS3', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding, Origin', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 d3fa3a50f1efd848dd27deb9ea2f30e0.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'HAN50-P2', 'X-Amz-Cf-Id': 'cRXmu4xzcPNJ44_pMfPkQXjfSKXYlLXyWtsDc3DcKz-lwTwLpc8YQw==', 'Age': '32'}